In [1]:
import torch
from torch import nn
from d2l import torch as d2l

#transposed convolution
def trans_conv(X,K):
    h,w=K.shape
    Y=torch.zeros(X.shape[0]+h-1,X.shape[1]+w-1)
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Y[i:i+h,j:j+w]+=X[i,j]*K
    return Y

In [4]:
#X、K必须设为浮点数tensor，否则下一个cell里的api用不了
#若为整数tensor，tconv.weight.data=K此句会报错
#RuntimeError：data set to a tensor that requires gradients must be floating point or complex dtype
#需要梯度的张量数据集必须是浮点或复数
X = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
trans_conv(X,K)

tensor([[ 0.,  0.,  1.],
        [ 0.,  4.,  6.],
        [ 4., 12.,  9.]])

In [ ]:
#使用pytorch的api实现transposed convolution
X,K=X.reshape(1,1,2,2),K.reshape(1,1,2,2)
tconv=nn.ConvTranspose2d(1,1,kernel_size=2,bias=False)
tconv.weight.data=K
tconv(X)

tensor([[[[ 0.,  0.,  1.],
          [ 0.,  4.,  6.],
          [ 4., 12.,  9.]]]], grad_fn=<ConvolutionBackward0>)

In [ ]:
#transposed convolution's padding
tconv=nn.ConvTranspose2d(1,1,kernel_size=2,padding=1,bias=False)
tconv.weight.data=K
tconv(X)

tensor([[[[4.]]]], grad_fn=<ConvolutionBackward0>)

In [8]:
#transposed convolution's stride
tconv=nn.ConvTranspose2d(1,1,kernel_size=2,stride=2,bias=False)
tconv.weight.data=K
tconv(X)

tensor([[[[0., 0., 0., 1.],
          [0., 0., 2., 3.],
          [0., 2., 0., 3.],
          [4., 6., 6., 9.]]]], grad_fn=<ConvolutionBackward0>)

In [ ]:
#昭示卷积和转置卷积之间的关系
#相同的超参数，二者对形状的变换可视为互逆
X=torch.rand(size=(1,10,16,16))
conv=nn.Conv2d(10,20,kernel_size=5,padding=2,stride=3)
tconv=nn.ConvTranspose2d(20,10,kernel_size=5,padding=2,stride=3)
tconv(conv(X)).shape==X.shape

True

In [ ]:
#下面两个cell用矩阵乘法来实现卷积
K = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
def kernel2matrix(K):
    k, W = torch.zeros(5), torch.zeros((4, 9))
    k[:2], k[3:5] = K[0, :], K[1, :]
    W[0, :5], W[1, 1:6], W[2, 3:8], W[3, 4:] = k, k, k, k
    return k,W

k,W = kernel2matrix(K)
k,W

(tensor([1., 2., 0., 3., 4.]),
 tensor([[1., 2., 0., 3., 4., 0., 0., 0., 0.],
         [0., 1., 2., 0., 3., 4., 0., 0., 0.],
         [0., 0., 0., 1., 2., 0., 3., 4., 0.],
         [0., 0., 0., 0., 1., 2., 0., 3., 4.]]))

In [17]:
X = torch.arange(9.0).reshape(3, 3)
Y = d2l.corr2d(X, K)
Y==torch.matmul(W,X.reshape(-1)).reshape(2,2)

tensor([[True, True],
        [True, True]])

In [18]:
#矩阵乘法实现转置卷积
Z=trans_conv(Y,K)
Z==torch.matmul(W.T,Y.reshape(-1)).reshape(3,3)

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])